In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
data = pd.read_excel("/content/Data Quality Assurance (Work Assignment Data).xlsx")

In [ ]:

# Get information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4440 entries, 0 to 4439
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   County                  4440 non-null   object 
 1   Field Manager           4440 non-null   object 
 2   Field Assistant         4401 non-null   object 
 3   Field Officer           4440 non-null   object 
 4   kcode                   4440 non-null   object 
 5   Tree Species            4440 non-null   object 
 6   Farmer FSEs Approved    4440 non-null   float64
 7   Zone Shamba Management  4440 non-null   object 
 8   Seedlings_Planted       4440 non-null   int64  
 9   gps_difference          4424 non-null   float64
 10  cycle_1_alive           4247 non-null   float64
 11  cycle_1_dead            4253 non-null   float64
 12  cycle_1_missing         4248 non-null   float64
 13  cycle_2_alive           4387 non-null   float64
 14  cycle_2_dead            4392 non-null   

Now we gonna answer the questions

Q1.**Data Quality Checks**

Q1(a): Missing Values

In [ ]:
# Check for missing values in the dataset
missing_values = data.isnull().sum()
print("Missing Values:\n", missing_values)


Missing Values:
 County                      0
Field Manager               0
Field Assistant            39
Field Officer               0
kcode                       0
Tree Species                0
Farmer FSEs Approved        0
Zone Shamba Management      0
Seedlings_Planted           0
gps_difference             16
cycle_1_alive             193
cycle_1_dead              187
cycle_1_missing           192
cycle_2_alive              53
cycle_2_dead               48
cycle_2_missing            48
dtype: int64


Q1 (b) Duplicates

In [ ]:
# Check for duplicate rows
duplicate_rows = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")


Number of duplicate rows: 0


Q1 (c) Cycle 1 reported seedlings (alive + dead + missing) > or < +/- 10 compared to seedlings planted

In [ ]:
# Check if Cycle 1 (alive + dead + missing) deviates by more than +/- 10 compared to seedlings planted
cycle_1_total = data['cycle_1_alive'] + data['cycle_1_dead'] + data['cycle_1_missing']
cycle_1_issue = (cycle_1_total < data['Seedlings_Planted'] - 10) | (cycle_1_total > data['Seedlings_Planted'] + 10)

# Display records where there's a Cycle 1 issue
cycle_1_errors = data[cycle_1_issue]
print(f"Number of Cycle 1 errors: {cycle_1_errors.shape[0]}")


Number of Cycle 1 errors: 684


Q1 (d) Cycle 2 reported seedlings (alive + dead + missing) > or < +/- 10 compared to seedlings planted

In [ ]:
# Check if Cycle 2 (alive + dead + missing) deviates by more than +/- 10 compared to seedlings planted
cycle_2_total = data['cycle_2_alive'] + data['cycle_2_dead'] + data['cycle_2_missing']
cycle_2_issue = (cycle_2_total < data['Seedlings_Planted'] - 10) | (cycle_2_total > data['Seedlings_Planted'] + 10)

# Display records where there's a Cycle 2 issue
cycle_2_errors = data[cycle_2_issue]
print(f"Number of Cycle 2 errors: {cycle_2_errors.shape[0]}")


Number of Cycle 2 errors: 1093


Q1 (e) Cycle 2 reported seedlings (alive + dead + missing) > or < +/- 10 compared to Cycle 1

In [ ]:
# Check if Cycle 2 deviates by more than +/- 10 compared to Cycle 1
cycle_2_vs_cycle_1_issue = (cycle_2_total < cycle_1_total - 10) | (cycle_2_total > cycle_1_total + 10)

# Display records where there's an issue between Cycle 1 and Cycle 2
cycle_2_vs_cycle_1_errors = data[cycle_2_vs_cycle_1_issue]
print(f"Number of Cycle 2 vs Cycle 1 errors: {cycle_2_vs_cycle_1_errors.shape[0]}")


Number of Cycle 2 vs Cycle 1 errors: 877


Q1 (f) Cycle 2 GPS > 150m difference

In [ ]:
# Check if GPS difference in Cycle 2 is greater than 150 meters
gps_issue = data['gps_difference'] > 150

# Display records where GPS difference exceeds 150m
gps_errors = data[gps_issue]
print(f"Number of GPS difference errors: {gps_errors.shape[0]}")


Number of GPS difference errors: 973


Q2: Final dataset with all error records

To combine all the identified issues (missing values, duplicates, Cycle 1, Cycle 2, Cycle 2 vs Cycle 1, and GPS issues), we'll create a final dataset with all records that have any of these errors.

In [ ]:
# Create flags for each error type
data['missing_values'] = data.isnull().any(axis=1)
data['duplicate'] = data.duplicated()
data['cycle_1_issue'] = cycle_1_issue
data['cycle_2_issue'] = cycle_2_issue
data['cycle_2_vs_cycle_1_issue'] = cycle_2_vs_cycle_1_issue
data['gps_issue'] = gps_issue

# Filter records with any errors
error_records = data[data[['missing_values', 'duplicate', 'cycle_1_issue', 'cycle_2_issue', 'cycle_2_vs_cycle_1_issue', 'gps_issue']].any(axis=1)]

# Display the error records
print(f"Number of records with any issues: {error_records.shape[0]}")
error_records.head()  # Display the first few error records


Number of records with any issues: 2138


,County,Field Manager,Field Assistant,Field Officer,kcode,Tree Species,Farmer FSEs Approved,Zone Shamba Management,Seedlings_Planted,gps_difference,...,cycle_1_missing,cycle_2_alive,cycle_2_dead,cycle_2_missing,missing_values,duplicate,cycle_1_issue,cycle_2_issue,cycle_2_vs_cycle_1_issue,gps_issue
0,Kilifi,Charles Chege Kariuki,George Ngala Chengo,Daidas Mwachiti,unk-2020-e517,Eucalyptus,1.0,zone-01-173,253,106.500196,...,36.0,192.0,2.0,0.0,False,False,True,True,True,False
1,Kilifi,Charles Chege Kariuki,Everlyn Nyevu Kahindi,Charles Wachira Mwema,unk-2020-d748,Eucalyptus,1.0,zone-01-173,253,7.214990,...,55.0,175.0,0.0,2.0,False,False,True,True,True,False
2,Kilifi,Charles Chege Kariuki,Jonathan Baraka Baya,Evelyn Ntinyari,unk-2020-c752,Eucalyptus,0.5,zone-01-173,125,104.060583,...,37.0,78.0,2.0,0.0,False,False,False,True,True,False
3,Kilifi,Charles Chege Kariuki,Everlyn Nyevu Kahindi,Charles Wachira Mwema,unk-2020-c754,Eucalyptus,0.5,zone-01-173,126,35.632934,...,10.0,100.0,3.0,7.0,False,False,False,True,True,False
4,Kilifi,Charles Chege Kariuki,Maurine Katana Faida,Charles Wachira Mwema,unk-2020-e486,Eucalyptus,0.5,zone-01-173,126,47.211515,...,16.0,100.0,0.0,2.0,False,False,False,True,True,False


#Export data to excel sheet

In [ ]:
# Export error records to Excel for use in Excel/Google Sheets
error_records.to_excel('data_quality_errors.xlsx', index=False)